In [13]:
# Google Drive Mound

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [14]:
# 데이터셋이 저장되는 기본 경로 설정

base_path = '/content/drive/MyDrive/Colab Notebooks/'

In [15]:
### Colab에서 MySQL을 사용하기 위한 환경설정 ###

# STEP 1) Install MySQL Server
!apt-get update
!apt-get install mysql-server -y

# STEP 2) Start MySQL Server
!service mysql start

# STEP 3) Set the Root Password
!sudo mysql -e "ALTER USER 'root'@'localhost' IDENTIFIED WITH mysql_native_password BY ''; FLUSH PRIVILEGES;"

# STEP 4) Install Python MySQL Connector
!pip install mysql-connector-python

# STEP 5) Connect and User MYSQL in Python
import mysql.connector

# Coonect to MySQL Server
conn = mysql.connector.connect(
    host = 'localhost',
    user = 'root',
    password = ''
)

cursor = conn.cursor()

# Create Database
cursor.execute("CREATE DATABASE IF NOT EXISTS testdb")
cursor.execute("USE testdb")

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:3 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading

In [16]:
# 필요한 모듈 로드

import pandas as pd
import numpy as np

from tabulate import tabulate

In [17]:
# Taks 1: Database Table Construction and ERD (30 Points)
# 1. HAR-UCI 데이터셋을 기반으로 MySQL에서 3개 테이블을 만들기
# - Users Table, Activities Table, SonsorReadings Table
# 2. ERD 설계
# 3. 데이터 삽입
# 4. 쿼리 결과 검증

# STEP 1) HAR-UCI 데이터셋을 기반으로 MySQL에서 3개 테이블 제작

cursor.execute("DROP TABLE IF EXISTS SensorReadings")
cursor.execute("DROP TABLE IF EXISTS Activities")
cursor.execute("DROP TABLE IF EXISTS Users")

create_users_query = """
 CREATE TABLE Users (
  user_id INT,
  PRIMARY KEY (user_id)
 );
"""

create_activities_query = """
 CREATE TABLE Activities (
  activity_id INT,
  activity_name VARCHAR(50) NOT NULL,
  PRIMARY KEY (activity_id)
 );
"""

create_sensorreadings_query = """
 CREATE TABLE SensorReadings (
  reading_id INT AUTO_INCREMENT,
  user_id INT,
  activity_id INT,
  AccelerometerX FLOAT,
  AccelerometerY FLOAT,
  AccelerometerZ FLOAT,
  GyroscopeX FLOAT,
  GyroscopeY FLOAT,
  GyroscopeZ FLOAT,
  PRIMARY KEY (reading_id),
  FOREIGN KEY (user_id) REFERENCES Users(user_id),
  FOREIGN KEY (activity_id) REFERENCES Activities(activity_id)
 );
"""

cursor.execute(create_users_query)
cursor.execute(create_activities_query)
cursor.execute(create_sensorreadings_query)

print("All Table Created") # 테이블 정상적으로 생성된 것 확인용

All Table Created


In [18]:
# STEP 2) ERD 설계
# 별도 플랫폼에서 진행 예정

In [19]:
# STEP 3) 데이터 삽입
# Users Table: /content/drive/MyDrive/Colab Notebooks/subject_records.txt에서 가져옴
# - 가져온 텍스트 파일을 user_id에 삽입

# Activities Table: /content/drive/MyDrive/Colab Notebooks/activities_labels.txt에서 가져옴
# - 가져온 데이터를 activity_id 와 activity_name에 삽입

# SensorReadings Table: /content/drive/MyDrive/Colab Notebooks/sensor_records.txt에서 가져옴
# tBodyAcc-mean()-X, tBodyAcc-mean()-Y, tBodyAcc-mean()-Z
# > AccelerometerX, AccelerometerY, AccelerometerZ로 변경
# tBodyGyro-mean()-X, tBodyGyro-mean()-Y, tBodyGyro-mean()-Z
# >  GyroscopeX, GyroscopeY, GyroscopeZ로 변경
# 각 activity_id는 /content/drive/MyDrive/Colab Notebooks/sensor_activity_records.txt에서 가져옴


# Users Table에 값 Insert 하기
# Google Drive에서 파일 가져오기
users_file_path = base_path + 'subject_records.txt'

with open(users_file_path, 'r') as f:
  user_ids = f.readlines()

user_ids = [int(uid.strip()) for uid in user_ids]

unique_user_ids = sorted(set(user_ids))

insert_users_query = """
 INSERT
 INTO Users (user_id)
 VALUES (%s);
"""

for uid in unique_user_ids:
  cursor.execute(insert_users_query, (uid,))


# Activities Table에 값 Insert 하기
activities_file_path = base_path + 'activity_records.txt'

with open(activities_file_path, 'r') as f:
  lines = f.readlines()

activity_data = []
for line in lines:
  parts = line.strip().split()
  activity_id = int(parts[0])
  activity_name = ' '.join(parts[1:])
  activity_data.append((activity_id, activity_name))

insert_activity_query = """
 INSERT
 INTO Activities (activity_id, activity_name)
 VALUES (%s, %s);
"""

for activity in activity_data:
  cursor.execute(insert_activity_query, activity)

# SensorReadings Table에 값 Insert 하기
features_path = base_path + 'features.txt'

target_features = [
    'tBodyAcc-mean()-X',
    'tBodyAcc-mean()-Y',
    'tBodyAcc-mean()-Z',
    'tBodyGyro-mean()-X',
    'tBodyGyro-mean()-Y',
    'tBodyGyro-mean()-Z'
]

with open(features_path, 'r') as f:
  lines = f.readlines()

feature_indices = []
for idx, line in enumerate(lines):
  feature_name = line.strip().split()[1]
  if feature_name in target_features:
    feature_indices.append(idx)

sensor_path = base_path + 'sensor_records.txt'
subject_path = base_path + 'subject_records.txt'
activity_path = base_path + 'sensor_activity_records.txt'

with open(sensor_path, 'r') as f:
  sensor_lines = f.readlines()

with open(subject_path, 'r') as f:
  user_ids = [int(line.strip()) for line in f]

with open(activity_path, 'r') as f:
  activity_ids = [int(line.strip()) for line in f]

assert len(sensor_lines) == len(user_ids) == len(activity_ids), "Data Length 일치 x"

sensor_insert_query = """
 INSERT
 INTO SensorReadings (
  user_id, activity_id,
  AccelerometerX, AccelerometerY, AccelerometerZ,
  GyroscopeX, GyroscopeY, GyroscopeZ
 )
 VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
"""

for i in range(len(sensor_lines)):
  sensor_values = sensor_lines[i].strip().split()
  selected_values = [float(sensor_values[j]) for j in feature_indices]

  row = (user_ids[i], activity_ids[i], *selected_values)
  cursor.execute(sensor_insert_query, row)

# 정상적으로 테이블이 형성되었는지 확인하기 위한 코드
cursor.execute("SELECT * FROM Users;")
users_rows = cursor.fetchall()

users_columns = [desc[0] for desc in cursor.description]
print(tabulate(users_rows, headers = users_columns, tablefmt = "psql"))

cursor.execute("SELECT * FROM Activities;")
activities_rows = cursor.fetchall()

activities_columns = [desc[0] for desc in cursor.description]
print(tabulate(activities_rows, headers = activities_columns, tablefmt = "psql"))

cursor.execute("SELECT * FROM SensorReadings;")
sensor_rows = cursor.fetchall()

sensor_columns = [desc[0] for desc in cursor.description]
print(sensor_columns)
# print(tabulate(sensor_rows, headers = sensor_columns, tablefmt = "psql").head())


+-----------+
|   user_id |
|-----------|
|         1 |
|         3 |
|         5 |
|         6 |
|         7 |
|         8 |
|        11 |
|        14 |
|        15 |
|        16 |
|        17 |
|        19 |
|        21 |
|        22 |
|        23 |
|        25 |
|        26 |
|        27 |
|        28 |
|        29 |
|        30 |
+-----------+
+---------------+--------------------+
|   activity_id | activity_name      |
|---------------+--------------------|
|             1 | WALKING            |
|             2 | WALKING_UPSTAIRS   |
|             3 | WALKING_DOWNSTAIRS |
|             4 | SITTING            |
|             5 | STANDING           |
|             6 | LAYING             |
+---------------+--------------------+
['reading_id', 'user_id', 'activity_id', 'AccelerometerX', 'AccelerometerY', 'AccelerometerZ', 'GyroscopeX', 'GyroscopeY', 'GyroscopeZ']


In [20]:
# STEP 4) 쿼리 결과 검증

def query_to_df(query):
  cursor.execute(query)
  result = cursor.fetchall()
  columns = [desc[0] for desc in cursor.description]
  return pd.DataFrame(result, columns = columns)

#-- Table Structure
# DESCRIBE Users;
# DESCRIBE Activities;
# DESCRIBE SensorReadings;

#-- Record Counts
# SELECT COUNT (*) FROM Users;
# SELECT COUNT (*) FROM Activities;
# SELECT COUNT (*) FROM SensorReadings;

describe_users_query = """
 DESCRIBE Users;
"""

describe_activities_query = """
 DESCRIBE Activities;
"""

describe_sensorreadings_query = """
 DESCRIBE SensorReadings;
"""

print(query_to_df(describe_users_query))
print(query_to_df(describe_activities_query))
print(query_to_df(describe_sensorreadings_query))

select_users_query = """
 SELECT COUNT(*)
 FROM Users;
"""

select_activities_query = """
 SELECT COUNT(*)
 FROM Activities;
"""

select_sensorreadings_query = """
 SELECT COUNT(*)
 FROM SensorReadings;
"""

print(query_to_df(select_users_query))
print(query_to_df(select_activities_query))
print(query_to_df(select_sensorreadings_query))

     Field Type Null  Key Default Extra
0  user_id  int   NO  PRI    None      
           Field         Type Null  Key Default Extra
0    activity_id          int   NO  PRI    None      
1  activity_name  varchar(50)   NO         None      
            Field   Type Null  Key Default           Extra
0      reading_id    int   NO  PRI    None  auto_increment
1         user_id    int  YES  MUL    None                
2     activity_id    int  YES  MUL    None                
3  AccelerometerX  float  YES         None                
4  AccelerometerY  float  YES         None                
5  AccelerometerZ  float  YES         None                
6      GyroscopeX  float  YES         None                
7      GyroscopeY  float  YES         None                
8      GyroscopeZ  float  YES         None                
   COUNT(*)
0        21
   COUNT(*)
0         6
   COUNT(*)
0      7352


In [21]:
# Task 2: SQL Query Practice
# Task 1에서 생성한 DB에 대해 SQL Queries를 실행.
# 각 쿼리는 테이블에 대한 이해 증진
# structure, joins, filtering, grouping, and aggregation
# 각 Question에 대해 적절한 SQL Query 작성 후 output에 대한 스크린샷과 설명을 PDF에 포함

# *GROUP BY: 데이터를 특정 기준으로 묶어서 그룹화 해줌
# - 집계 함수 (COUNT, SUM, AVG, MAX, MIN 등)과 함께 사용
# 행들을 하나 이상의 컬럼 값을 기준으로 그룹화 하여, 각 그룹에 대해 집계 연산을 수행


# Q1. How many sensor records exist per user? (Please use 'GROUP BY')
q1_query = """
 SELECT user_id, COUNT(*) AS record_count
 FROM SensorReadings
 GROUP BY user_id;
"""

print(query_to_df(q1_query))

# Q2. How many sensor records per activity (Please use GROUP BY)
q2_query = """
 SELECT activity_id, COUNT(*) AS activity_record_count
 FROM SensorReadings
 GROUP BY activity_id;
"""

print(query_to_df(q2_query))

# Q3. How many times did user 1 perform each activity?
q3_query = """
 SELECT activity_id, COUNT(*) AS activity_count
 FROM SensorReadings
 WHERE user_id = 1
 GROUP BY activity_id
 ORDER BY activity_id ASC;
"""

print(query_to_df(q3_query))

# Q4. How many times did user 3 perform each activity?
q4_query = """
 SELECT activity_id, COUNT(*) AS activity_count
 FROM SensorReadings
 WHERE user_id = 3
 GROUP BY activity_id
 ORDER BY activity_id ASC;
"""

print(query_to_df(q4_query))

# Q5. What is the average AccelerometerX value for each activity?
q5_query = """
 SELECT activity_id, AVG(AccelerometerX) AS avg_accelerometer_x
 FROM SensorReadings
 GROUP BY activity_id
"""

print(query_to_df(q5_query))

# Q6. What is the average GyroscopeY value for each activity?
q6_query = """
 SELECT activity_id, AVG(GyroscopeY) AS avg_gyroscope_y
 FROM SensorReadings
 GROUP BY activity_id
"""

print(query_to_df(q6_query))

    user_id  record_count
0         1           347
1         3           341
2         5           302
3         6           325
4         7           308
5         8           281
6        11           316
7        14           323
8        15           328
9        16           366
10       17           368
11       19           360
12       21           408
13       22           321
14       23           372
15       25           409
16       26           392
17       27           376
18       28           382
19       29           344
20       30           383
   activity_id  activity_record_count
0            1                   1226
1            2                   1073
2            3                    986
3            4                   1286
4            5                   1374
5            6                   1407
   activity_id  activity_count
0            1              95
1            2              53
2            3              49
3            4              47
4      

In [22]:
# Task3: Acitivity Prediction (40 points)
# /content/drive/MyDrive/Colab Notebooks/records_for_analysis.txt
# - tBodyAcc-mean()-X, tBodyAcc-mean()-Y, tBodyAcc-mean()-Z
# - tBodyGyro-mean()-X, tBodyGyro-mean()-Y, tBody
# 새로운 30개의 센서 데이터 레코드에 대해 어떤 활동에 해당하는 지 예측하는 것
# 기존 DB (SensorReadings Table)에 저장된 활동별 평균 센서 벡터와 비교해 가장 유사한 활동 예측

# Task 3 종료 후 7 columns가 있는 task3_result_pd를 생성

# STEP 1) SensorReadings 테이블에서 activity_id 별 평균값 구하기
avg_vector_query = """
 SELECT activity_id,
 AVG(AccelerometerX) AS acc_x,
 AVG(AccelerometerY) AS acc_y,
 AVG(AccelerometerZ) AS acc_z,
 AVG(GyroscopeX) AS gyr_x,
 AVG(GyroscopeY) AS gyr_y,
 AVG(GyroscopeZ) AS gyr_z
 FROM SensorReadings
 GROUP BY activity_id;
"""
print(query_to_df(avg_vector_query))
activity_means_df = query_to_df(avg_vector_query)

# STEP 2) record_for_analysis.txt 데이터 가져오기
analysis_file_path = base_path + 'records_for_analysis_new_250602.txt'

column_names = [
    'tBodyAcc-mean()-X',
    'tBodyAcc-mean()-Y',
    'tBodyAcc-mean()-Z',
    'tBodyGyro-mean()-X',
    'tBodyGyro-mean()-Y',
    'tBodyGyro-mean()-Z'
]

analysis_df = pd.read_csv(analysis_file_path, sep = '\s+', header=0, names=column_names)

print(analysis_df.head())

# STEP 3) 예측: 각 행에 대해 DB로부터 얻은 활동별 평균 벡터들과 비교
# - Euclidean Distance를 사용해 가장 가까운 활동을 찾음

def euclidean_distance(vec1, vec2):
  return np.sqrt(np.sum((vec1 - vec2) ** 2))

predicted_activities = []

for i in range(len(analysis_df)):
  test_vector = analysis_df.iloc[i].values.astype(float)
  min_dist = float('inf')
  predicted_label = None

  for j in range(len(activity_means_df)):
    activity_vector = activity_means_df.iloc[j, 1:].values.astype(float)
    dist = euclidean_distance(test_vector, activity_vector)

    if dist < min_dist:
      min_dist = dist
      predicted_label = activity_means_df.iloc[j]['activity_id']

  predicted_activities.append(predicted_label)

# STEP 4) 예측 결과 저장
task3_result_pd = analysis_df.copy()
task3_result_pd['Predicted Activity'] = predicted_activities

print(task3_result_pd)


   activity_id     acc_x     acc_y     acc_z     gyr_x     gyr_y     gyr_z
0            1  0.276260 -0.017768 -0.108884 -0.031253 -0.071313  0.084194
1            2  0.261930 -0.026647 -0.120424  0.020924 -0.096946  0.049120
2            3  0.288169 -0.016370 -0.105860 -0.076662 -0.055910  0.093925
3            4  0.273449 -0.012143 -0.106581 -0.037831 -0.072803  0.078474
4            5  0.279294 -0.016123 -0.107330 -0.026323 -0.066075  0.078778
5            6  0.269191 -0.018345 -0.107169 -0.016761 -0.091661  0.127148
   tBodyAcc-mean()-X  tBodyAcc-mean()-Y  tBodyAcc-mean()-Z  \
0           0.269163          -0.048915          -0.151687   
1           0.273417          -0.005495          -0.098009   
2           0.272671           0.001040          -0.112612   
3           0.317956           0.004834          -0.079667   
4           0.281497          -0.019456          -0.107823   

   tBodyGyro-mean()-X  tBodyGyro-mean()-Y  tBodyGyro-mean()-Z  
0            0.008569           -0.112